Эта автоматизированная программа собирает данные об объявлениях с сайта недвижимости bn.ru. 

Основная идея: bn.ru за один поисковый запрос показывает лишь 510 объявлений, поэтому, чтобы собрать все данные, мы делим весь ценовой диапазон на отрезки таким образом, что в каждом промежутке находилось не более 510 квартир.

Результат выдается в виде json-файла full_db(today_day-today_month-today_year).json. Она также берет за базу вчерашний файл: full_db(yesterday_day-yesterday_month-yesterday_year).json, чтобы писать историю, не собирать некоторые не меняющиеся данные и т.д..

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import requests
from multiprocessing.dummy import Pool as ThreadPool
from bs4 import BeautifulSoup
import random
import time
import datetime
import math
import json
import datetime as dt

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
pool = ThreadPool(50) # этот ThreadPool будет использоваться для отправки параллельных запросов.

In [3]:
# функции, нужные для сбора информации со страниц поисковика
def check_amount(lower, upper, limit = 510):
    '''Проверяет, что квартир в ценовом диапазоне от lower до upper меньше limit'''
    r = requests.get(f'https://www.bn.ru/kvartiry-vtorichka/?priceFrom={lower}&priceTo={upper}')
    soup = BeautifulSoup(r.text, 'html.parser')
    amount = int(soup.find('strong').contents[0])
    if amount <= limit:
        return True, amount
    else:
        return False, amount


def check_bounds(price_bounds, limit = 510):
    '''Проверяет, что в списке ограничений price_bounds все ограничения проходят проверку check_amount и что сумма кол-ва всех квартир, 
    лежащих внутри ограничений price_bounds равна сумме объявлений на сайте'''
    r = requests.get(f'https://www.bn.ru/kvartiry-vtorichka')
    soup = BeautifulSoup(r.text, 'html.parser')
    amount = int(soup.find('strong').contents[0])
    summa = 0
    for interval in price_bounds:
        l, u, _ = interval
        flag, count = check_amount(l, u)
        if not flag:
            print(l, u)
            return False
        summa += count
    if summa == amount:
        return True
    else:
        print("Counted: ", summa, " Total: ", amount)
        return False


def calc_borders():
    '''Вычисляет список подходящих ограничений на цены'''
    answer = []
    lower = 0
    upper = 3000000
    step = 100000
    print('Посчитаны:')
    while upper < 20 * 10 ** 6:
        state, _ = check_amount(lower, upper)
        while state:
            if upper > 20 * 10 ** 6:
                break
            upper += step
            state, _ = check_amount(lower, upper)
        upper -= step
        _, amount = check_amount(lower, upper - 1)
        answer.append((lower, upper - 1, amount))
        print((lower, upper - 1, amount)) # печатаем для контроля за выполнением
        lower = upper
    # --- const-значения для ускорения подсчета. Внутри этих ограничений кол-во квартир <510 и чаще всего не меняется. 
    _, amount = check_amount(lower, 24999999)
    answer.append((lower, 24999999, amount))
    print((lower, 24999999, amount))
    _, amount = check_amount(25000000, 29999999)
    answer.append((25000000, 29999999, amount))
    print((25000000, 29999999, amount))
    _, amount = check_amount(30000000, 39999999)
    answer.append((30000000, 39999999, amount))
    print((30000000, 39999999, amount))
    _, amount = check_amount(40000000, 5785000000)
    answer.append((40000000, 5785000000, amount))
    print((40000000, 5785000000, amount))             
    return answer # [(lower_bound1, upper_bound1, amount_of_ads_between_lower_and_upper1), (lower_bound2, upper_bound2, amount_of_ads_between_lower_and_upper2), ...]


def get_ids_from_page(soup):
    '''Собирает со страницы все айди объявлений'''
    divs = soup.find_all("div", {"class": "catalog-item__id"})
    ids = [str(i.contents[0]) for i in divs]
    return ids


def get_prices_from_page(soup):
    '''Собирает со страницы все цены объявлений'''
    divs = soup.find_all("div", {"class": "catalog-item__price"})
    prices = [str(i.contents[0]) for i in divs]
    return prices


def get_additional_info1(soup):
    '''Собирает со страницы информацию о статусе объявлений (проводятся ли онлайн-показы, является ли автор объявления - собственником)'''
    result = []
    divs = soup.find_all("div", {"class": "catalog-item__mark-container"})
    for div in divs:
        part = []
        if div.find('span', {"class": "catalog-item__mark-item catalog-item__mark-item-online"}):
            part.append(str(div.find('span', {"class": "catalog-item__mark-item catalog-item__mark-item-online"}).contents[0]))
        if div.find('a'):
            part.append(str(div.find('a').contents[0])) 
        if len(part) != 0:
            result.append(sorted(part))
        else:
            result.append(None)
    return result


def get_additional_info2(soup):
    '''Собирает со страницы информацию о продвижении объявлений: 
    top - объявление выводится в верх поиска и окрашивается; 
    up - объявление выводится в верх поиска;
    top - объявление окрашивается;
    day - объявлению выдан статус объекта дня;
    warn - объявление выдан статус "подозрительного".'''
    result = []
    divs = soup.find_all("div", {"class": "catalog-item__vas-container"})
    for div in divs:
        part = []
        if div.find('div', {"class": "catalog-item__vas-icon catalog-item__vas-icon-top"}):
            part.append('top')
        if div.find('div', {"class": "catalog-item__vas-icon catalog-item__vas-icon-up"}):
            part.append('up') 
        if div.find('div', {"class": "catalog-item__vas-icon catalog-item__vas-icon-color"}):
            part.append('color')
        if div.find('div', {"class": "catalog-item__vas-icon catalog-item__vas-icon-object-day"}):
            part.append('day')
        if div.find('div', {"class": "catalog-item__vas-icon catalog-item__vas-icon-warn"}):
            part.append('warn')
        if len(part) != 0:
            result.append(sorted(part))
        else:
            result.append(None)
    return result


def get_dates_from_page(soup):
    '''Собирает со страницы информацию о дате изменения объявлений'''
    spans = soup.find_all('span', {'class':'catalog-item__date-value'})
    dates = [str(i.contents[0]) for i in spans]
    return dates


def get_addresses_from_page(soup):
    '''Собирает со страницы информацию об адресах объявлений'''
    divs = soup.find_all('div', {'class':'catalog-item__address'})
    addresses = [str(i.contents[0]) if len(i.contents) != 0 else None for i in divs]
    return addresses    


def parse_info_from_price(bounds):
    '''Собирает значения get_ids_from_page, get_prices_from_page, get_additional_info1, get_additional_info2, get_addresses_from_page
    для всех квартир, которые находятся в bounds = [lower_bounds, upper_bounds]'''
    global pool
    ids_from_price = []
    prices_from_price = []
    info1 = []
    info2 = []
    dates_from_price = []
    address_from_price = []
    l, u = bounds
    url = f'https://www.bn.ru/kvartiry-vtorichka/?priceFrom={l}&priceTo={u}'
    s = requests.Session()
    r = s.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    td = soup.find('strong')
    amount = int(td.contents[0])
    npages = math.ceil(amount/30)
    page_ids = get_ids_from_page(soup)
    page_prices = get_prices_from_page(soup)
    page_info1 = get_additional_info1(soup)
    page_info2 = get_additional_info2(soup)
    page_dates = get_dates_from_page(soup)
    page_addresses = get_addresses_from_page(soup)

    ids_from_price.extend(page_ids)
    prices_from_price.extend(page_prices)
    info1.extend(page_info1)
    info2.extend(page_info2)
    dates_from_price.extend(page_dates)
    address_from_price.extend(page_addresses)
    urls = [f'https://www.bn.ru/kvartiry-vtorichka/?priceFrom={l}&priceTo={u}&page={page}' for page in range(2, npages + 1)]
    answers = pool.map(requests.get, urls)
    for r in answers:
        soup = BeautifulSoup(r.text, 'html.parser')
        page_ids = get_ids_from_page(soup)
        page_prices = get_prices_from_page(soup)
        page_info1 = get_additional_info1(soup)
        page_info2 = get_additional_info2(soup)
        page_dates = get_dates_from_page(soup)
        page_addresses = get_addresses_from_page(soup)
        ids_from_price.extend(page_ids)
        prices_from_price.extend(page_prices)
        info1.extend(page_info1)
        info2.extend(page_info2)
        dates_from_price.extend(page_dates)
        address_from_price.extend(page_addresses)
    return ids_from_price, prices_from_price, info1, info2, dates_from_price, address_from_price


def parse_info_from_bounds(list_of_bounds):
    '''Собирает значени parse_info_from_price для всех ограничений list_of_bounds = [bounds1, bounds2, ...]'''
    ids = []
    prices = []
    info1 = []
    info2 = []
    dates = []
    addresses = []
    for elem in list_of_bounds:
        l, u, _ = elem
        bounds = (l, u)
        price_ids, price_prices, price_info1, price_info2, price_dates, price_addresses = parse_info_from_price(bounds)
        ids.extend(price_ids)
        prices.extend(price_prices)
        info1.extend(price_info1)
        info2.extend(price_info2)
        dates.extend(price_dates)
        addresses.extend(price_addresses)
    return ids, prices, info1, info2, dates, addresses


def collect_info(list_of_bounds):
    '''Собирает информацию об объявлениях с сайта. Проходимся по сайту два раза, т.к поисковик иногда выдает одинаковые квартиры'''
    result1 = dict()
    result2 = dict()
    r = requests.get(f'https://www.bn.ru/kvartiry-vtorichka')
    soup = BeautifulSoup(r.text, 'html.parser')
    amount = int(soup.find('strong').contents[0])
    id1, price1, info1_1, info2_1, dates1, addresses1 = parse_info_from_bounds(list_of_bounds)
    assert (len(id1) == len(price1)) and (len(id1) == len(info1_1)) and (len(id1) == len(info2_1)) and (len(id1) == len(dates1)) and (len(id1) == len(addresses1))
    for i in range(len(id1)):
        result1[id1[i]] = {'price': price1[i], 'info1': info1_1[i], 'info2': info2_1[i], 'dates': dates1[i], 'address': addresses1[i]}
    id2, price2, info1_2, info2_2, dates2, addresses2 = parse_info_from_bounds(list_of_bounds[::-1])
    for i in range(len(id2)):
        result2[id2[i]] = {'price': price2[i], 'info1': info1_2[i], 'info2': info2_2[i], 'dates': dates2[i], 'address': addresses2[i]}
    result = {**result1, **result2}
    return result

# --------------------------------
# функции, нужные для сбора информации со страниц квартир
def parse_flat_page(soup):
    '''Собирает информацию о квартире со страницы объявления'''
    name = soup.find_all("h1", {"class": "object-2019__header-headline"})
    price = soup.find_all("div", {"class": "object-2019__header-price"})
    how_selled = soup.find_all("div", {"class": "object-2019__header-price-unit"})
    metro = soup.find_all("span", {"class": "object__header-metro-name"})
    address = soup.find_all("a", {"class": "object-2019__header-address", "href": "#map-yandex"})
    date = soup.find_all("div", {"class": "object__id"})  
    author = soup.find_all("div", {"class": "object__user"})      
    characteristics = soup.find_all("div", {"class": "object__param-item-value"})
    desc = soup.find_all("div", {"class": "object__comment"})
    close_metro = soup.find_all("span", {"class": "object__transport-metro-name"})
    close_railway = soup.find_all("span", {"class": "object__transport-station-name"})
    transport_dist = soup.find_all("span", {"class": "object__transport-distance"})
    flat_info = {'name': name, 'price': price, 'how_selled': how_selled, 'metro': metro, 'address': address, 'date': date, 
                'author': author, 'characteristics': characteristics, 'desc': desc, 
                'close_metro': close_metro, 'close_railway': close_railway, 'transport_dist': transport_dist}
    return flat_info


def parse_flats(list_of_ids):
    '''Принимает на вход список айди объявлений и выдает информацию о квартирах в формате словарь словарей'''
    global pool
    all_flats = dict()
    chunked_ids = list(divide_chunks(list_of_ids, 100))
    counter = 0
    for chunk in chunked_ids:
        urls = [f'https://www.bn.ru/detail/flats/{id}/' for id in chunk]
        answers = pool.map(requests.get, urls)
        for i in range(len(chunk)):
            soup = BeautifulSoup(answers[i].text, 'html.parser')
            all_flats[chunk[i]] = delete_tags(parse_flat_page(soup))
        counter += 1
    return all_flats


def delete_tags(dicti):
    '''Удаляет из словаря parse_flat_page все теги'''
    result = dict()
    # address
    if dicti['address']:
        result['address'] = str(dicti['address'][0].contents[0])
    else:
        result['address'] = None
    # author
    if dicti['author']:
        if str(dicti['author'][0]) == 'частное':
            result['company'] = str(dicti['author'][0].find('span').contents[0])
            result['agent'] = None
        else:
            result['company'] = str(dicti['author'][0].find('span').contents[0])
            if len(dicti['author'][0].find_all('a')) == 2:
                result['agent'] = str(dicti['author'][0].find_all('a')[1].contents[0])
            else:
                result['agent'] = None
    # char
    if dicti['characteristics']:
        result['characteristics'] = [str(i.contents[0]) if len(i.contents) != 0 else None for i in dicti['characteristics']]
    else:
        result['characteristics'] = None
    # close_metro 
    if dicti['close_metro']:
        result['close_metro'] = [str(i.contents[0]) if len(i.contents) != 0 else None for i in dicti['close_metro']]
    else:
        result['close_metro'] = None
    # close_railway
    if dicti['close_railway']:
        result['close_railway'] = [str(i.contents[0]) if len(i.contents) != 0 else None for i in dicti['close_railway']]
    else:
        result['close_railway'] = None
    #date
    if dicti['date']:
        result['date'] = str(dicti['date'][0].find('span').contents[0])
    else:
        result['date'] = None
    # desc 
    if dicti['desc']:
        if len(dicti['desc'][0].contents) != 0:
            strings = [str(i) for i in dicti['desc'][0].contents]
            result['desc'] = ''.join(strings)
        else:
            result['desc'] = None
    else:
        result['desc'] = None
    # how_selled
    if dicti['how_selled']:
        result['how_selled'] = str(dicti['how_selled'][0].contents[2])
    else:
        result['how_selled'] = None
    # metro
    if dicti['metro']:
        result['metro'] = str(dicti['metro'][0].contents[0])
    else:
        result['metro'] = None        
    # name
    if dicti['name']:
        result['name'] = str(dicti['name'][0].contents[0])
    else:
        result['name'] = None
    # price
    if dicti['price']:
        result['price'] = str(dicti['price'][0].contents[0])
    else:
        result['price'] = None
    # transport_dist
    if dicti['transport_dist']:
        result['transport_dist'] = [str(i.contents[0]) if len(i.contents) != 0 else None for i in dicti['transport_dist']]
    else:
        result['transport_dist'] = None
    return result

# --------------------------------
# вспомогательные функции 
def normal_round(n):
    '''арифметическое округление'''
    if n - math.floor(n) < 0.5:
        return math.floor(n)
    return math.ceil(n)


def divide_chunks(l, n):
    '''Разбивает массив на фрагменты'''
    for i in range(0, len(l), n): 
        yield l[i:i + n]


def send_requests(list_of_ids):
    '''отправляет запросы ко всем id из списка list_of_ids'''
    result = dict()
    chunked_ids = list(divide_chunks(list_of_ids, 100))
    counter = 0
    for chunk in chunked_ids:
        urls = [f'https://www.bn.ru/detail/flats/{id}/' for id in chunk]
        answers = pool.map(requests.get, urls)
        for i in range(len(chunk)):
            result[chunk[i]] = answers[i].status_code
    return result

Загружаем с диска вчерашнюю базу

In [4]:
# даты
today = dt.datetime.today()
today_day, today_month, today_year = today.day, today.month, today.year
yesterday = today - dt.timedelta(days=1)
yesterday_day, yesterday_month, yesterday_year = yesterday.day, yesterday.month, yesterday.year

In [5]:
dir = 'drive/My Drive/Colab Notebooks/CSC/Projects/REZ_8/'
with open(dir + f'full_db({yesterday_day}-{yesterday_month}-{yesterday_year}).json') as json_file:
    flats = json.load(json_file)

Используем уже заготовленный список ценовых ограничений. Проверяем его. Если он не проходит тест, используем функцию calc_borders, чтобы пересчитать ограничения.

In [6]:
price_bounds = [(0, 2899999, 478), (2900000, 3599999, 433), (3600000, 3899999, 366), (3900000, 4099999, 283), (4100000, 4299999, 331), 
                 (4300000, 4599999, 450), (4600000, 4799999, 320), (4800000, 4999999, 346), (5000000, 5199999, 303), (5200000, 5399999, 424), 
                 (5400000, 5499999, 248), (5500000, 5699999, 394), (5700000, 5899999, 301), (5900000, 6099999, 377), (6100000, 6299999, 363), 
                 (6300000, 6499999, 326), (6500000, 6699999, 404), (6700000, 6899999, 277), (6900000, 7199999, 414), (7200000, 7499999, 390), 
                 (7500000, 7799999, 406), (7800000, 8099999, 392), (8100000, 8499999, 362), (8500000, 8899999, 425), (8900000, 9399999, 471), 
                 (9400000, 9899999, 488), (9900000, 10499999, 382), (10500000, 11299999, 459), (11300000, 11899999, 402), (11900000, 12599999, 474), 
                 (12600000, 13799999, 486), (13800000, 15399999, 484), (15400000, 17799999, 490), (17800000, 19999999, 325), (20000000, 24999999, 397), 
                 (25000000, 29999999, 277), (30000000, 39999999, 329), (40000000, 5785000000, 431)]

In [ ]:
all_ok = check_bounds(price_bounds)

Собираем данные из поисковика

In [8]:
ids_info = collect_info(price_bounds)
today_ids = set(i.split(' ')[1] for i in ids_info.keys())
print('кол-во собранных объявлений сегодня: ', len(today_ids))
to_check = list(set(flats.keys()) - today_ids)
print('кол-во объявлений, которые нужно проверить: ', len(to_check)) # как я уже говорил ранее поисковик bn.ru часто шалит и не показывает квартиры
# поэтому, чтобы удостовериться, что объявление действительно отсутствует на сайте, я кидаю к ним запросы и смотрю на код ответа.
checked_ids = send_requests(to_check)
existing_past_ids = []
for key in checked_ids.keys():
    if checked_ids[key] == 200:
        existing_past_ids.append(key)
print('кол-во объявлений, которые поисковик скрыл, но они присутствуют на сайте:', len(existing_past_ids))
new_ids = list(today_ids - set(flats.keys()))
print('кол-во новых объявлений:', len(new_ids))

кол-во собранных объявлений сегодня:  15070
кол-во объявлений, которые нужно проверить:  10831
кол-во объявлений, которые поисковик скрыл, но они присутствуют на сайте: 170
кол-во новых объявлений: 1476


Собираем данные со страниц квартиры. Обходим только новые объявления и прошлые объявления, которые скрыл поисковик.

In [9]:
parsed_past = parse_flats(existing_past_ids)
parsed_new = parse_flats(new_ids)

Вносим новые данные в базу и сохраняем файл

In [10]:
for id in flats.keys():
    key = 'id ' + id
    if key in ids_info:
        flats[id]['update_history'].append((ids_info[key]['dates'], f'{today_day}-{today_month}-{today_year}'))
        flats[id]['info1_history'].append((ids_info[key]['info1'], f'{today_day}-{today_month}-{today_year}'))
        flats[id]['info2_history'].append((ids_info[key]['info2'], f'{today_day}-{today_month}-{today_year}'))
        flats[id]['price_history'].append((ids_info[key]['price'], f'{today_day}-{today_month}-{today_year}'))
        flats[id]['address_history'].append((ids_info[key]['address'], f'{today_day}-{today_month}-{today_year}'))
    elif id in parsed_past:
        flats[id]['update_history'].append((parsed_past[id]['date'], f'{today_day}-{today_month}-{today_year}'))
        flats[id]['info1_history'].append((None, f'{today_day}-{today_month}-{today_year}'))
        flats[id]['info2_history'].append((None, f'{today_day}-{today_month}-{today_year}'))
        flats[id]['price_history'].append((parsed_past[id]['price'], f'{today_day}-{today_month}-{today_year}'))
        flats[id]['address_history'].append((parsed_past[id]['address'], f'{today_day}-{today_month}-{today_year}'))
    else:
        flats[id]['update_history'].append((None, f'{today_day}-{today_month}-{today_year}'))
        flats[id]['info1_history'].append((None, f'{today_day}-{today_month}-{today_year}'))
        flats[id]['info2_history'].append((None, f'{today_day}-{today_month}-{today_year}'))
        flats[id]['price_history'].append((None, f'{today_day}-{today_month}-{today_year}'))
        flats[id]['address_history'].append((None, f'{today_day}-{today_month}-{today_year}'))         
for id in parsed_new.keys():
    key = 'id ' + id
    flats[id] = parsed_new[id]
    flats[id]['update_history'] = [(ids_info[key]['dates'], f'{today_day}-{today_month}-{today_year}')]
    flats[id]['info1_history'] = [(ids_info[key]['info1'], f'{today_day}-{today_month}-{today_year}')]
    flats[id]['info2_history'] = [(ids_info[key]['info2'],f'{today_day}-{today_month}-{today_year}')]
    flats[id]['price_history'] = [(ids_info[key]['price'],f'{today_day}-{today_month}-{today_year}')]
    flats[id]['address_history'] = [(ids_info[key]['address'], f'{today_day}-{today_month}-{today_year}')]

In [11]:
for id in random.sample(flats.keys(), 3):
    print(id, ': ', flats[id])

3672709 :  {'address': 'Санкт-Петербург, Выборгский район, Парголово, Заречная ул.', 'company': 'Петербургская недвижимость', 'agent': None, 'characteristics': ['21.10 кв.м.', '14.00 кв.м.', None, None, None, 'Нет', None, 'Совмещенный', None, None, None, '15/25', 'Кирпично-Монолитный', '2020', None, None, None, None, None], 'close_metro': ['Парнас', 'Проспект Просвещения'], 'close_railway': ['Шувалово', 'Парголово'], 'date': 'сегодня 00:08', 'desc': 'Жилой комплекс Жили-Были, собственность (1 взрослый собственник), прямая продажа квартиры без обременений, подходит под ипотеку, квартира-студия с удобной планировкой и качественной полной отделкой от застройщика, площадь прихожей - 3,8 кв.м., совмещенный санузел с установленной душевой кабиной, раковиной и унитазом, остекленная лоджия - 5 кв.м. (не включена в общую площадь квартиры) с видом на Шуваловский парк, станция метро Парнас (20 минут прогулки пешком), удобная автотранспортная доступность с Ольгинской дороги, закрытый внутренний дв

Атрибуты:
*   address - адрес квартиры;
*   company - компания, подавшая объявление;
*   agent - агент, ответственный за объявление;
*   characteristics - различные характеристики квартиры;
*   close_metro - ближайшее метро;
*   close_railway - ближайшие ЖД станции;
*   desc - описание в объявлении;
*   how_selled - как квартира продается;
*   metro - станция метро рядом с квартирой;
*   name - название объявления (кол-во комнат);
*   transport_dist - расстояние до ближайших станций метро и ЖД;
*   update_history - история времени обновления объявления;
*   price_history - история цен объявления;
*   info1_history - история характеристик объявления (статус собственника и онлайн-показы);
*   info1_history - история продвижения объявления;





In [12]:
dir = 'drive/My Drive/Colab Notebooks/CSC/Projects/REZ_8/'
with open(dir + f'full_db({today_day}-{today_month}-{today_year}).json', 'w') as f:
    json.dump(flats, f)